# Explain what does PyTorch 2.0 do to your code

PyTorch 2.0 looks like magic to many researchers, as they dynamically translate the bytecode for you. Many people don't know Python bytecode, so they don't know how do the translated code look like. Fortunately, with `depyf`, we can clearly illustrate it for you.

# Runtime requirements

Please use **Jupyter Lab**, not **Jupyter Notebook**.

If you didn't install the following dependencies, please **restart the jupyter lab server after installing these dependencies.**

If you have already installed them, skip the next cell.

In [ ]:
# Install dependencies and enable widgets for display
!pip install ipywidgets
!jupyter labextension install @jupyter-widgets/jupyterlab-manager
!jupyter labextension enable widgetsnbextension

# Run some code with `torch.compile`, and use `eager` backend

Note that we use `eager` backend so that the compiled subgraph runs in eager mode, and then we can easily get its code.

In [1]:
import torch
@torch.compile(backend="eager")
def toy_example(a, b):
    x = a / (torch.abs(a) + 1)
    if b.sum() < 0:
        b = b * -1
    return x * b

for _ in range(100):
    toy_example(torch.randn(10), torch.randn(10))

# Import the library for display

In [8]:
%run lib_explain.py

# Explain everything you are curious about, with the magical `explain` function

In [9]:
explain(toy_example)

# transformed source code:

<IPython.core.display.JSON object>

# source code of referenced function:

<details>
  <summary>__resume_at_34_1</summary>

  ```python
  def <resume in toy_example>(b, x):
      b = b * -1
      return x * b

  ```
</details>
<details>
  <summary>__compiled_fn_3</summary>

  ```python
  def forward(self, L_b_ : torch.Tensor, L_x_ : torch.Tensor):
      l_b_ = L_b_
      l_x_ = L_x_
      mul = l_x_ * l_b_;  l_x_ = l_b_ = None
      return (mul,)

  ```
</details>
<details>
  <summary>__compiled_fn_0</summary>

  ```python
  def forward(self, L_a_ : torch.Tensor, L_b_ : torch.Tensor):
      l_a_ = L_a_
      l_b_ = L_b_
      abs_1 = torch.abs(l_a_)
      add = abs_1 + 1;  abs_1 = None
      x = l_a_ / add;  l_a_ = add = None
      __temp_2 = l_b_.sum();  l_b_ = None
      lt = __temp_2 < 0;  __temp_2 = None
      return (x, lt)

  ```
</details>
<details>
  <summary>cache_code:0</summary>

  ```python
  def toy_example(a, b):
      __temp_22 = __compiled_fn_0(a, b)
      x = __temp_22[0]
      if __temp_22[1]:
          return __resume_at_34_1(b, x)
      return __resume_at_42_2(b, x)

  ```
</details>
<details>
  <summary>cache_code:2</summary>

  ```python
  def <resume in toy_example>(b, x):
      return __compiled_fn_3(b, x)[0]

  ```
</details>
<details>
  <summary>toy_example</summary>

  ```python
  def toy_example(a, b):
      x = a / (torch.abs(a) + 1)
      if b.sum() < 0:
          b = b * -1
      return x * b

  ```
</details>
<details>
  <summary>cache_code:1</summary>

  ```python
  def <resume in toy_example>(b, x):
      return __compiled_fn_4(b, x)[0]

  ```
</details>
<details>
  <summary>__resume_at_42_2</summary>

  ```python
  def <resume in toy_example>(b, x):
      return x * b

  ```
</details>
<details>
  <summary>__compiled_fn_4</summary>

  ```python
  def forward(self, L_b_ : torch.Tensor, L_x_ : torch.Tensor):
      l_b_ = L_b_
      l_x_ = L_x_
      b = l_b_ * -1;  l_b_ = None
      mul_1 = l_x_ * b;  l_x_ = b = None
      return (mul_1,)

  ```
</details>


# Going further, we can directly explain referenced functions!

In [10]:
explain(__resume_at_42_2)

# transformed source code:

<IPython.core.display.JSON object>

# source code of referenced function:

<details>
  <summary>__compiled_fn_3</summary>

  ```python
  def forward(self, L_b_ : torch.Tensor, L_x_ : torch.Tensor):
      l_b_ = L_b_
      l_x_ = L_x_
      mul = l_x_ * l_b_;  l_x_ = l_b_ = None
      return (mul,)

  ```
</details>
<details>
  <summary>cache_code:3</summary>

  ```python
  def <resume in toy_example>(b, x):
      return __compiled_fn_3(b, x)[0]

  ```
</details>
<details>
  <summary>__resume_at_42_2</summary>

  ```python
  def <resume in toy_example>(b, x):
      return x * b

  ```
</details>


In [11]:
explain(__compiled_fn_4)

# transformed source code:

<IPython.core.display.JSON object>

# source code of referenced function:

<details>
  <summary>forward</summary>

  ```python
  def forward(self, L_b_, L_x_):
      l_b_ = L_b_
      l_x_ = L_x_
      b = l_b_ * -1
      l_b_ = None
      mul_1 = l_x_ * b
      l_x_ = None
      b = None
      return mul_1,

  ```
</details>


# General notice

If you find `compiled_code_entries` is not empty for a function, it probably means PyTorch does not execute its source code. PyTorch will tries to examine the code entries, and execute the code whose guarding conditions are satisfied.